In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch
import os
from dotenv import load_dotenv
import uuid
import json
load_dotenv()

True

In [2]:

# pprint(os.getenv("ELASTIC_SEARCH_API_KEY"))

In [3]:
## JSON printer

def printer(data):
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)

In [4]:
es = Elasticsearch(os.getenv("ELASTIC_SEARCH_URL"), api_key=os.getenv("ELASTIC_SEARCH_API_KEY"))

client_info = es.info()

pprint("Connected to elastic Search")
printer(client_info.body)

'Connected to elastic Search'
{
    "name": "serverless",
    "cluster_name": "fb96fd8be3e24d4c8a75e6aaf5802bf6",
    "cluster_uuid": "R11pXxymTzym4PV5vRNTYg",
    "version": {
        "number": "8.11.0",
        "build_flavor": "serverless",
        "build_type": "docker",
        "build_hash": "00000000",
        "build_date": "2023-10-31",
        "build_snapshot": false,
        "lucene_version": "9.7.0",
        "minimum_wire_compatibility_version": "8.11.0",
        "minimum_index_compatibility_version": "8.11.0"
    },
    "tagline": "You Know, for Search"
}


In [22]:
es.indices.delete(index="myindex", ignore_unavailable=True)

# Not going to work on serverless
# es.indices.create(index="myindex", settings={
#     "index": {
#         "number_of_shards": 3,
#         "number_of_replica": 2
#     }
# })

index_name = "myindex"

es.indices.create(index=index_name)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'myindex'})

In [6]:
from datetime import datetime

pprint(datetime.now())

datetime.datetime(2025, 8, 31, 17, 28, 59, 68266)


In [7]:
# random_uuid = uuid.uuid4()
# # print(random_uuid)

first_document = {
    "document_id" : uuid.uuid4(),
    "title" : "Test Document 1",
    "author" : "Uday Yadav",
    "date" : datetime.now()
}

response_object = es.index(index=index_name, document=first_document)
printer(response_object.body)

{
    "_index": "myindex",
    "_id": "GK8sAZkB-JJed6hy1CkK",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 1,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 0,
    "_primary_term": 1
}


In [8]:
import json

dummy_data = json.load(open("./data/dummy_data.json"))
printer(dummy_data)

[
    {
        "title": "Sample Title 1",
        "text": "This is the first sample document text.",
        "created_on": "2024-09-22"
    },
    {
        "title": "Sample Title 2",
        "text": "Here is another example of a document.",
        "created_on": "2024-09-24"
    },
    {
        "title": "Sample Title 3",
        "text": "The content of the third document goes here.",
        "created_on": "2024-09-24"
    }
]


In [23]:
response_id = []

In [24]:
def insert_document(insert_document):
    return es.index(index=index_name, document=insert_document)


for document in dummy_data:
    document["document_id"] = uuid.uuid4()
    response = insert_document(document)
    printer(response.body)
    response_id.append(response["_id"])

{
    "_index": "myindex",
    "_id": "Ia8zAZkB-JJed6hyjSkn",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 1,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 0,
    "_primary_term": 1
}
{
    "_index": "myindex",
    "_id": "Iq8zAZkB-JJed6hyjinB",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 1,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 0,
    "_primary_term": 1
}
{
    "_index": "myindex",
    "_id": "I68zAZkB-JJed6hyjyne",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 1,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 1,
    "_primary_term": 1
}


In [11]:
printer(response_id)

[
    "Ga8sAZkB-JJed6hy1ik_",
    "Gq8sAZkB-JJed6hy1yng",
    "G68sAZkB-JJed6hy2SkC"
]


In [12]:
## How to delete document

try:
    delete_response = es.delete(index=index_name, id=response_id[0])
    printer(delete_response)
except Exception as e:
    print(f"Document Doesnt exists: {e}")

Document Doesnt exists: Object of type ObjectApiResponse is not JSON serializable


In [13]:
## How to create document

try:
    response_document = es.get(index=index_name, id=response_id[0])
    printer(response_document.body)
except Exception as e:
    print(f"Document Doesnt exists: {e}")

Document Doesnt exists: NotFoundError(404, "{'_index': 'myindex', '_id': 'Ga8sAZkB-JJed6hy1ik_', 'found': False}")


In [14]:
## How to count documents
try:
    count_response = es.count(index=index_name)
    printer(count_response.body)
except Exception as e:
    print(f"Error counting documents: {e}")

{
    "count": 2,
    "_shards": {
        "total": 3,
        "successful": 3,
        "skipped": 0,
        "failed": 0
    }
}


In [15]:
## Write a simple query

date_query = {
    "range": {
        "created_on" : {
            "gte": "2024-09-23"
        }
    }
}

query_response = es.search(index=index_name, query=date_query)
printer(query_response.body)

{
    "took": 1,
    "timed_out": false,
    "_shards": {
        "total": 3,
        "successful": 3,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 2,
            "relation": "eq"
        },
        "max_score": 1.0,
        "hits": [
            {
                "_index": "myindex",
                "_id": "Gq8sAZkB-JJed6hy1yng",
                "_score": 1.0,
                "_source": {
                    "title": "Sample Title 2",
                    "text": "Here is another example of a document.",
                    "created_on": "2024-09-24",
                    "document_id": "e553f6b8-4b6f-477f-a8b4-265022aeff3f"
                }
            },
            {
                "_index": "myindex",
                "_id": "G68sAZkB-JJed6hy2SkC",
                "_score": 1.0,
                "_source": {
                    "title": "Sample Title 3",
                    "text": "The content of the third document goes here

In [16]:
response = es.indices.exists(index=index_name)
printer(response.body)

response = es.exists(index=index_name, id=response_id[2])
printer(response.body)

true
true


In [25]:
# update document

original_document =   {
    "title": "Sample Title 4",
    "text": "The content of the fourth document goes here",
    "created_on": "2024-09-25"
}

inserted_documented = insert_document(original_document)
printer(inserted_documented.body)

document_id = inserted_documented["_id"]
print("New document id " + str(document_id))

updated_response = es.update(index=index_name, id=document_id, script={
    "source" : "ctx._source.title = params.title",
    "params" : {
        "title": "Sample Title 5"
    }
})

printer(updated_response.body)

updated_document = es.get(index=index_name, id=document_id)
printer(updated_document.body)

# es.delete(index=index_name, id=document_id)

{
    "_index": "myindex",
    "_id": "JK80AZkB-JJed6hyoiku",
    "_version": 1,
    "result": "created",
    "_shards": {
        "total": 1,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 1,
    "_primary_term": 1
}
New document id JK80AZkB-JJed6hyoiku
{
    "_index": "myindex",
    "_id": "JK80AZkB-JJed6hyoiku",
    "_version": 2,
    "result": "updated",
    "_shards": {
        "total": 1,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 2,
    "_primary_term": 1
}
{
    "_index": "myindex",
    "_id": "JK80AZkB-JJed6hyoiku",
    "_version": 2,
    "_seq_no": 2,
    "_primary_term": 1,
    "found": true,
    "_source": {
        "title": "Sample Title 5",
        "text": "The content of the fourth document goes here",
        "created_on": "2024-09-25"
    }
}


In [ ]:
random_uuid = uuid.uuid4()

update_response = es.update(index=index_name, id=document_id, script={
    "source": f"ctx._source.document_id = '{random_uuid}'"
})

printer(updated_response.body)
updated_document = es.get(index=index_name, id=updated_response["_id"])
printer(updated_document.body)


{
    "_index": "myindex",
    "_id": "JK80AZkB-JJed6hyoiku",
    "_version": 2,
    "result": "updated",
    "_shards": {
        "total": 1,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 2,
    "_primary_term": 1
}
{
    "_index": "myindex",
    "_id": "JK80AZkB-JJed6hyoiku",
    "_version": 3,
    "_seq_no": 3,
    "_primary_term": 1,
    "found": true,
    "_source": {
        "title": "Sample Title 5",
        "text": "The content of the fourth document goes here",
        "created_on": "2024-09-25",
        "document_id": "e61fd956-a8dc-4557-8bee-1252f0739d01"
    }
}
